# Image classification using transfer learning - solutions

This notebook is very similiar to the other notebook used for teaching in this class, however in this notebook we will be performing *transfer learning*. This means instead of training a CNN from scratch, for most of the weights of the network we will be using pre-trained weights trained on another dataset. This is surprisingly effective, especially when using pre-trained weights from a very large and general purpose dataset like imagenet. 

Because we are not training all the weights from scratch, we can use significantly larger images in this notebook while still being able to train a model quickly.

Just like the other notebook we will be using our own custom datasets for training this classifier.

## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Load your dataset

To train a classification model, we need a dataset of images where different categories of images are organised into classes. This is as simple as putting images of different things into different folder.
i.e:
```
my_dataset/clouds/cloud_0001.jpg
my_dataset/clouds/cloud_0002.jpg
..
my_dataset/candy_floss/cfloss_0001.jpg
my_dataset/candy_floss/cfloss_0002.jpg
```

There are plenty of classification datasets on the internet but it's more fun and more of a challenge to train a classifier on a dataset that we have made ourselves. Why don't you take the image dataset you made in the last session and use that! You may need to add an extra category for classification if you only had one category images previously. You can refer back to [last weeks instructions](https://moodle.arts.ac.uk/mod/page/view.php?id=954147) if you get stuck. All you need to do is put some new images in a different folder, and then put both those folders into a folder for the whole dataset. You don't need to worry about resizing all the images for this task as keras will take care of image resizing for us.

Once you have got your dataset, change the path in the `dataset_path` variable to the path to your dataset.s

In [ ]:
image_size = (150, 150)
batch_size = 128
num_classes = 2
dataset_path = 'path/to/dataset'

train = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset='training',
    seed=1,
    image_size=image_size,
    batch_size=batch_size,
)

validation = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset='validation',
    seed=1,
    image_size=image_size,
    batch_size=batch_size,
)

## Visualize the data

This will show you some samples from your data. The numbers are the class indexes used by the model for classification.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

## Load existing model

In [ ]:
#Change MobileNet for EfficientNet
base_model = keras.applications.EfficientNetB0(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
if num_classes == 2:
    outputs = keras.layers.Dense(1)(x)
else:
    outputs = keras.layers.Dense(num_classes)(x)
model = keras.Model(inputs, outputs)

## Train the model

In [ ]:
epochs = 10
if num_classes == 2:
    loss_function = "binary_crossentropy"
else:
    loss_function = "categorical_crossentropy"

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=loss_function,
    metrics=["accuracy"],
)
model.fit(
    train,
    epochs=epochs,
    validation_data=validation,
)

### Optional: save the model
This is how we save the weights of the model for savekeeping

In [ ]:
model.save('classifer_transfer_learned.model')

### Next steps
Look at the [available models on the keras API website](https://keras.io/api/applications/#available-models) and try changing the code in this notebook to work with a different pretrained model. **Warning** some bigger CNN models will take a lot longer to train than the MobileNet model used in this notebook. You may want to make sure your laptop is plugged in before training some of these models!

### Bonus exercise
Look at [this Keras tutorial for transfer learning](https://keras.io/guides/transfer_learning/), read through the code and see if you can integrate the portion of the code that performs fine-tuning on the model (training all of the layers) after fine-tuning into this code notebook. Then perform a second round of training with your transfer learned model to further fine-tune it to your dataset.

In [ ]:
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=loss_function,
    metrics=["accuracy"],
)

epochs = 10
model.fit(train, epochs=epochs, validation_data=validation)

In [ ]:
model.save('classifer_fine_tuned.model')